# 웹 크롤링(Web Crawling) 완전 정복 🕷️

## 학습 목표
- 웹 크롤링의 개념과 원리 이해
- BeautifulSoup과 Selenium 라이브러리 활용
- 정적/동적 웹페이지 크롤링 실습
- 실제 웹사이트 데이터 수집 및 분석

## 목차
1. 웹 크롤링 기초 개념
2. HTML 구조와 CSS 선택자
3. BeautifulSoup을 이용한 정적 크롤링
4. Selenium을 이용한 동적 크롤링
5. 실전 프로젝트

## 1. 웹 크롤링 기초 개념

### 1.1 웹 크롤링이란?
- **정의**: 웹페이지에서 원하는 데이터를 자동으로 수집하는 기술
- **용도**: 데이터 분석, 가격 비교, 뉴스 수집, 시장 조사 등
- **종류**:
  - **정적 크롤링**: HTML이 고정된 웹페이지 (requests + BeautifulSoup)
  - **동적 크롤링**: JavaScript로 생성되는 콘텐츠 (Selenium)

### 1.2 크롤링 vs 스크래핑
- **크롤링**: 웹페이지를 탐색하며 링크를 따라가는 과정
- **스크래핑**: 특정 웹페이지에서 데이터를 추출하는 과정

### 1.3 주의사항 ⚠️
- **robots.txt** 확인: 웹사이트의 크롤링 정책 준수
- **요청 간격**: 서버에 부하를 주지 않도록 적절한 딜레이
- **법적/윤리적 고려**: 저작권, 개인정보보호법 준수

In [ ]:
# 필수 라이브러리 설치
# !pip install requests beautifulsoup4 lxml selenium pandas matplotlib seaborn

# 라이브러리 임포트
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from urllib.parse import urljoin, urlparse
import matplotlib.pyplot as plt
import seaborn as sns

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

print("라이브러리 임포트 완료!")

## 2. HTML 구조와 CSS 선택자

### 2.1 HTML 기본 구조
```html
<!DOCTYPE html>
<html>
<head>
    <title>페이지 제목</title>
</head>
<body>
    <div class="container">
        <h1 id="title">메인 제목</h1>
        <p class="content">내용</p>
        <ul>
            <li>항목 1</li>
            <li>항목 2</li>
        </ul>
    </div>
</body>
</html>
```

### 2.2 CSS 선택자
- **태그 선택자**: `h1`, `p`, `div`
- **클래스 선택자**: `.content`, `.container`
- **ID 선택자**: `#title`
- **자식 선택자**: `div > p` (직접 자식)
- **후손 선택자**: `div p` (모든 하위)
- **속성 선택자**: `[href]`, `[class="content"]`

In [ ]:
# BeautifulSoup 기초 실습
html_content = """
<html>
<head><title>크롤링 실습</title></head>
<body>
    <div class="container">
        <h1 id="main-title">웹 크롤링 실습</h1>
        <p class="description">BeautifulSoup을 이용한 HTML 파싱</p>
        <ul class="item-list">
            <li class="item">항목 1</li>
            <li class="item">항목 2</li>
            <li class="item">항목 3</li>
        </ul>
        <div class="info">
            <span class="price">15,000원</span>
            <span class="discount">20% 할인</span>
        </div>
    </div>
</body>
</html>
"""

# BeautifulSoup 객체 생성
soup = BeautifulSoup(html_content, 'html.parser')

# 다양한 선택자 사용법
print("=== BeautifulSoup 선택자 실습 ===")
print(f"제목: {soup.find('h1').text}")
print(f"설명: {soup.find('p', class_='description').text}")
print(f"ID로 선택: {soup.find(id='main-title').text}")

# 여러 요소 선택
items = soup.find_all('li', class_='item')
print(f"\n항목들:")
for i, item in enumerate(items, 1):
    print(f"  {i}. {item.text}")

# CSS 선택자 사용
price = soup.select_one('.info .price').text
discount = soup.select_one('.info .discount').text
print(f"\n가격 정보:")
print(f"  가격: {price}")
print(f"  할인: {discount}")

## 3. BeautifulSoup을 이용한 정적 크롤링

### 3.1 실제 웹사이트 크롤링
네이버 뉴스 헤드라인을 수집하는 실습을 진행합니다.

In [ ]:
# 네이버 뉴스 헤드라인 크롤링 예제
def crawl_naver_news():
    """네이버 뉴스 메인페이지에서 헤드라인 수집"""
    url = "https://news.naver.com/"
    
    # User-Agent 설정 (서버가 차단하지 않도록)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        # 웹페이지 요청
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # HTTP 에러 체크
        
        # BeautifulSoup으로 파싱
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # 뉴스 헤드라인 추출 (선택자는 실제 사이트 구조에 따라 변경 필요)
        headlines = []
        
        # 메인 뉴스 영역에서 제목 추출
        news_titles = soup.find_all('a', class_='cluster_text_headline')
        
        for title in news_titles[:10]:  # 상위 10개만
            headlines.append({
                'title': title.text.strip(),
                'link': urljoin(url, title.get('href', ''))
            })
        
        return headlines
        
    except requests.RequestException as e:
        print(f"요청 중 오류 발생: {e}")
        return []

# 뉴스 헤드라인 수집 실행
print("=== 네이버 뉴스 헤드라인 크롤링 ===")
news_data = crawl_naver_news()

if news_data:
    for i, news in enumerate(news_data, 1):
        print(f"{i}. {news['title']}")
        print(f"   링크: {news['link'][:50]}...")
        print()
else:
    print("뉴스 데이터를 가져올 수 없습니다.")

## 4. Selenium을 이용한 동적 크롤링

### 4.1 Selenium이란?
- **정의**: 웹 브라우저를 자동화하는 도구
- **장점**: JavaScript가 실행된 후의 DOM 상태 접근 가능
- **사용 사례**: SPA(Single Page Application), AJAX 로딩, 버튼 클릭 등

### 4.2 ChromeDriver 설정
```bash
# ChromeDriver 다운로드 필요
# https://chromedriver.chromium.org/
```

### 4.3 주요 기능
- **요소 찾기**: `find_element()`, `find_elements()`
- **동작 수행**: 클릭, 텍스트 입력, 스크롤
- **대기**: `implicitly_wait()`, `WebDriverWait()`
- **스크린샷**: `save_screenshot()`

In [ ]:
# Selenium 기본 사용법
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

def setup_driver():
    """ChromeDriver 설정"""
    chrome_options = Options()
    # chrome_options.add_argument('--headless')  # 헤드리스 모드 (브라우저 창 안 띄움)
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
    
    try:
        driver = webdriver.Chrome(options=chrome_options)
        driver.implicitly_wait(10)  # 10초 대기
        return driver
    except Exception as e:
        print(f"드라이버 설정 오류: {e}")
        print("ChromeDriver가 설치되어 있는지 확인하세요.")
        return None

# 구글 검색 자동화 예제
def google_search_example(query):
    """구글에서 특정 키워드 검색"""
    driver = setup_driver()
    if not driver:
        return
    
    try:
        # 구글 메인페이지 접속
        driver.get("https://www.google.com")
        
        # 검색창 찾기 및 검색어 입력
        search_box = driver.find_element(By.NAME, "q")
        search_box.send_keys(query)
        search_box.send_keys(Keys.RETURN)
        
        # 검색 결과 대기
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "search"))
        )
        
        # 검색 결과 제목들 추출
        results = driver.find_elements(By.CSS_SELECTOR, "h3")
        
        print(f"=== '{query}' 검색 결과 (상위 5개) ===")
        for i, result in enumerate(results[:5], 1):
            print(f"{i}. {result.text}")
        
        # 스크린샷 저장
        driver.save_screenshot('google_search_result.png')
        print("\n스크린샷이 'google_search_result.png'로 저장되었습니다.")
        
    except Exception as e:
        print(f"오류 발생: {e}")
    finally:
        driver.quit()

# 실행 (주석 해제하여 사용)
# google_search_example("Python 크롤링")
print("Selenium 설정 완료! 위의 주석을 해제하여 구글 검색 예제를 실행하세요.")

## 5. 실전 프로젝트: 쇼핑몰 상품 정보 크롤링

### 5.1 프로젝트 개요
- **목표**: 온라인 쇼핑몰에서 상품 정보 수집
- **수집 데이터**: 상품명, 가격, 평점, 리뷰 수 등
- **활용**: 가격 비교, 시장 분석, 데이터 시각화

In [ ]:
# 쇼핑몰 상품 정보 크롤링 실습
import random

def crawl_product_info():
    """가상의 쇼핑몰 상품 정보 크롤링 (예시)"""
    
    # 실제 사이트 대신 가상 데이터 생성 (교육용)
    products = []
    categories = ['노트북', '스마트폰', '태블릿', '이어폰', '마우스']
    brands = ['삼성', 'LG', '애플', '소니', '로지텍']
    
    for i in range(20):
        product = {
            'id': i + 1,
            'name': f"{random.choice(brands)} {random.choice(categories)} {i+1}",
            'price': random.randint(50000, 2000000),
            'rating': round(random.uniform(3.0, 5.0), 1),
            'reviews': random.randint(10, 1000),
            'category': random.choice(categories)
        }
        products.append(product)
    
    return products

# 상품 데이터 생성
products_data = crawl_product_info()

# DataFrame으로 변환
df_products = pd.DataFrame(products_data)
print("=== 크롤링된 상품 정보 ===")
print(df_products.head(10))

# 기본 통계 정보
print(f"\n=== 데이터 요약 ===")
print(f"총 상품 수: {len(df_products)}")
print(f"평균 가격: {df_products['price'].mean():,.0f}원")
print(f"평균 평점: {df_products['rating'].mean():.2f}")
print(f"카테고리별 상품 수:")
print(df_products['category'].value_counts())

In [ ]:
# 크롤링 데이터 시각화
plt.figure(figsize=(15, 10))

# 1. 카테고리별 상품 수
plt.subplot(2, 3, 1)
df_products['category'].value_counts().plot(kind='bar', color='skyblue')
plt.title('카테고리별 상품 수')
plt.xticks(rotation=45)

# 2. 가격 분포
plt.subplot(2, 3, 2)
plt.hist(df_products['price'], bins=15, color='lightgreen', alpha=0.7, edgecolor='black')
plt.title('상품 가격 분포')
plt.xlabel('가격 (원)')
plt.ylabel('상품 수')

# 3. 평점 분포
plt.subplot(2, 3, 3)
plt.hist(df_products['rating'], bins=10, color='orange', alpha=0.7, edgecolor='black')
plt.title('상품 평점 분포')
plt.xlabel('평점')
plt.ylabel('상품 수')

# 4. 카테고리별 평균 가격
plt.subplot(2, 3, 4)
category_price = df_products.groupby('category')['price'].mean().sort_values(ascending=False)
category_price.plot(kind='bar', color='coral')
plt.title('카테고리별 평균 가격')
plt.xticks(rotation=45)
plt.ylabel('평균 가격 (원)')

# 5. 가격 vs 평점 산점도
plt.subplot(2, 3, 5)
plt.scatter(df_products['price'], df_products['rating'], alpha=0.6, color='purple')
plt.title('가격 vs 평점')
plt.xlabel('가격 (원)')
plt.ylabel('평점')

# 6. 리뷰 수 vs 평점
plt.subplot(2, 3, 6)
plt.scatter(df_products['reviews'], df_products['rating'], alpha=0.6, color='red')
plt.title('리뷰 수 vs 평점')
plt.xlabel('리뷰 수')
plt.ylabel('평점')

plt.tight_layout()
plt.show()

# 상관관계 분석
correlation = df_products[['price', 'rating', 'reviews']].corr()
print("\n=== 수치 데이터 상관관계 ===")
print(correlation)

## 6. 크롤링 모범 사례 및 팁 💡

### 6.1 기술적 팁
- **User-Agent 설정**: 실제 브라우저처럼 보이게 하기
- **요청 간격 조절**: `time.sleep()` 사용으로 서버 부하 방지
- **예외 처리**: `try-except` 구문으로 안정성 확보
- **데이터 검증**: 수집된 데이터의 유효성 검사

### 6.2 법적/윤리적 고려사항
- **robots.txt 확인**: `사이트URL/robots.txt` 접확인
- **이용약관 준수**: 웹사이트의 이용약관 검토
- **개인정보 보호**: 개인정보는 수집하지 않기
- **상업적 이용 주의**: 저작권 및 데이터 소유권 고려

### 6.3 성능 최적화
- **세션 재사용**: `requests.Session()` 활용
- **병렬 처리**: `threading` 또는 `asyncio` 사용
- **캐싱**: 중복 요청 방지
- **효율적인 선택자**: CSS 선택자 최적화

### 6.4 디버깅 팁
- **개발자 도구 활용**: F12로 HTML 구조 분석
- **단계별 테스트**: 작은 부분부터 점진적 개발
- **로그 기록**: 진행 상황 및 오류 추적
- **데이터 저장**: CSV, JSON 등으로 중간 결과 저장

In [ ]:
# 크롤링 유틸리티 함수들
import csv
import json
from datetime import datetime

class CrawlingUtils:
    """크롤링에 유용한 유틸리티 함수들"""
    
    @staticmethod
    def save_to_csv(data, filename=None):
        """데이터를 CSV 파일로 저장"""
        if filename is None:
            filename = f"crawling_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        
        if isinstance(data, pd.DataFrame):
            data.to_csv(filename, index=False, encoding='utf-8-sig')
        else:
            df = pd.DataFrame(data)
            df.to_csv(filename, index=False, encoding='utf-8-sig')
        
        print(f"데이터가 '{filename}'에 저장되었습니다.")
    
    @staticmethod
    def save_to_json(data, filename=None):
        """데이터를 JSON 파일로 저장"""
        if filename is None:
            filename = f"crawling_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        
        print(f"데이터가 '{filename}'에 저장되었습니다.")
    
    @staticmethod
    def clean_text(text):
        """텍스트 정리 (공백, 특수문자 제거)"""
        if text:
            # 앞뒤 공백 제거
            text = text.strip()
            # 연속된 공백을 하나로 변경
            text = re.sub(r'\s+', ' ', text)
            # 특수 문자 제거 (필요에 따라 수정)
            text = re.sub(r'[^\w\s가-힣]', '', text)
        return text
    
    @staticmethod
    def extract_numbers(text):
        """텍스트에서 숫자만 추출"""
        if text:
            numbers = re.findall(r'\d+', text)
            return ''.join(numbers)
        return ''
    
    @staticmethod
    def validate_url(url):
        """URL 유효성 검사"""
        try:
            result = urlparse(url)
            return all([result.scheme, result.netloc])
        except:
            return False

# 유틸리티 함수 사용 예제
print("=== 크롤링 유틸리티 함수 예제 ===")

# 텍스트 정리 예제
sample_text = "  삼성 갤럭시 S21   (5G)  "
clean_text = CrawlingUtils.clean_text(sample_text)
print(f"원본: '{sample_text}'")
print(f"정리 후: '{clean_text}'")

# 숫자 추출 예제
price_text = "가격: 1,200,000원"
numbers = CrawlingUtils.extract_numbers(price_text)
print(f"가격 텍스트: '{price_text}'")
print(f"숫자만: '{numbers}'")

# URL 검증 예제
urls = ["https://www.example.com", "invalid-url", "http://google.com"]
for url in urls:
    is_valid = CrawlingUtils.validate_url(url)
    print(f"URL: {url} -> 유효: {is_valid}")

# 데이터 저장 예제 (위에서 생성한 products_data 사용)
print(f"\n=== 데이터 저장 예제 ===")
CrawlingUtils.save_to_csv(df_products, "products_sample.csv")
CrawlingUtils.save_to_json(products_data, "products_sample.json")

## 7. 수업 정리 및 연습 문제 📝

### 7.1 학습 내용 요약
1. **웹 크롤링 기초**: 정적 vs 동적, HTML 구조 이해
2. **BeautifulSoup**: 정적 웹페이지에서 데이터 추출
3. **Selenium**: 동적 웹페이지 자동화 및 데이터 수집
4. **데이터 처리**: 수집된 데이터 정리 및 저장
5. **시각화**: 크롤링 데이터를 차트로 표현
6. **모범 사례**: 윤리적, 기술적 고려사항

### 7.2 연습 문제 🚀

#### 연습 1: 날씨 정보 크롤링
- 기상청 또는 날씨 사이트에서 오늘의 날씨 정보 수집
- 온도, 습도, 미세먼지 등의 정보 추출
- 데이터프레임으로 정리하여 출력

#### 연습 2: 주식 정보 수집
- 네이버 금융에서 코스피 상위 10개 종목 정보 수집
- 종목명, 현재가, 등락률 데이터 추출
- 시각화를 통해 등락률 분석

#### 연습 3: 도서 정보 크롤링
- 온라인 서점에서 베스트셀러 목록 수집
- 책 제목, 저자, 가격, 평점 정보 추출
- 장르별 평균 가격 분석

#### 연습 4: 부동산 정보 분석
- 부동산 사이트에서 특정 지역 매물 정보 수집
- 가격, 면적, 위치 정보 추출
- 면적당 가격 계산 및 시각화

### 7.3 추가 학습 자료
- **공식 문서**: 
  - BeautifulSoup: https://www.crummy.com/software/BeautifulSoup/bs4/doc/
  - Selenium: https://selenium-python.readthedocs.io/
- **실습 사이트**: 
  - http://quotes.toscrape.com/ (크롤링 연습용)
  - https://httpbin.org/ (HTTP 요청 테스트)
- **고급 주제**: 
  - Scrapy 프레임워크
  - 비동기 크롤링 (aiohttp)
  - 분산 크롤링 시스템

In [ ]:
# 마지막 실습: 간단한 크롤링 챌린지
print("🎯 크롤링 수업 완료! 🎯")
print("\n=== 수업에서 다룬 주요 기술들 ===")
print("✅ HTML 파싱 (BeautifulSoup)")
print("✅ CSS 선택자 활용")
print("✅ 웹 브라우저 자동화 (Selenium)")
print("✅ 데이터 정리 및 저장")
print("✅ 시각화를 통한 데이터 분석")
print("✅ 크롤링 모범 사례")

print("\n=== 다음 단계 학습 추천 ===")
print("🔸 API 활용법 학습")
print("🔸 대용량 데이터 처리 (Pandas 고급)")
print("🔸 데이터베이스 연동")
print("🔸 머신러닝 데이터 전처리")
print("🔸 웹 애플리케이션 개발")

print("\n" + "="*50)
print("    🚀 Happy Coding! 🚀")
print("="*50)

# 수업 완료 기념 간단한 통계
lesson_stats = {
    'topics_covered': 7,
    'code_examples': 10,
    'libraries_used': ['requests', 'beautifulsoup4', 'selenium', 'pandas', 'matplotlib'],
    'practical_projects': 3
}

print(f"\n📊 수업 통계:")
for key, value in lesson_stats.items():
    print(f"   {key}: {value}")

# 학습자를 위한 체크리스트
checklist = [
    "웹 크롤링의 기본 개념을 이해했나요?",
    "BeautifulSoup으로 HTML을 파싱할 수 있나요?",
    "Selenium으로 브라우저를 자동화할 수 있나요?",
    "수집한 데이터를 정리하고 시각화할 수 있나요?",
    "크롤링 시 주의사항을 알고 있나요?"
]

print(f"\n✅ 학습 체크리스트:")
for i, item in enumerate(checklist, 1):
    print(f"   {i}. {item}")

print(f"\n🎓 수고하셨습니다! 크롤링 마스터가 되셨네요!")